In [13]:
import glob
import os.path
from concurrent.futures import ProcessPoolExecutor
from pathlib import Path

from PIL import Image
from tqdm.notebook import tqdm


class ImagesGenerator:
  def __init__(self, image_generator):
      self._image_generator = image_generator

  def generate_images(self):
    params = list(self._processing_params_generator())
    with ProcessPoolExecutor() as executor:
        results = list(tqdm(executor.map(self._process_image, params), total=len(params)))
    return results

  def _processing_params_generator(self):
      for diagonal in self._image_generator.diagonals_to_ppi_scale_width_height:
        source_image_paths = self._image_generator.generate_source_image_paths(diagonal)
        output_directory = Path(os.path.join(
          self._image_generator.destination_images_root, diagonal))
        output_directory.mkdir(parents=True, exist_ok=True)

        for source_image_path in source_image_paths:
            yield diagonal, output_directory, source_image_path

  def _process_image(self, params):
      diagonal, output_directory, source_image_path = params
      image = self._image_generator.generate_image(source_image_path, diagonal)
      destination_image_path = os.path.join(
          output_directory, os.path.basename(source_image_path))
      image.save(destination_image_path)

      return image


class AppStorePreviewImageGenerator:
  buttons = {
      'gamepad': {
          'path': '../Bundle/JSDPad/dPad-None@2x.png',
          'offset': (20, -20-150),  # +x = left, -x = right, +y = top, -y = bottom
          'size': (150, 150),
          'opacity': round(.125 * 255),
      },
      'esc': {
          'path': '../Screenshots for app store/esc.png',
          'offset': (-20-32, 28),
          'size': (32, 15),
          'opacity': 255,
      },
      'menu': {
          'path': '../Screenshots for app store/menu.png',
          'offset': (-20-74, -20-259),
          'size': (74, 259),
          'opacity': 255,
      },
      'tab': {
          'path': '../Screenshots for app store/tab.png',
          'offset': (94, 28),
          'size': (32, 15),
          'opacity': 255,
      },
      'btab': {
          'path': '../Screenshots for app store/btab.png',
          'offset': (20, 28),
          'size': (41, 15),
          'opacity': 255,
      },
  }
  
  source_images_root = '../Screenshots for app store/sources'
  destination_images_root = '../Screenshots for app store/generated'
  diagonals_to_ppi_scale_width_height = {
      '5.5': (401, 2, 2208, 1242),
      '6.5': (458, 3, 2688, 1242),
      '12.9': (264, 2, 2732, 2048),
  }


  def generate_image(self, source_image_path, diagonal):
      bg_image = Image.open(source_image_path)
      dpi, scale, *size = self.diagonals_to_ppi_scale_width_height[diagonal]

      for name, data in self.buttons.items():
          self._process_button(bg_image, data, size, scale)
      bg_image = bg_image.convert('RGB')
      return bg_image

  def generate_source_image_paths(self, diagonal):
    source_image_paths = glob.glob(os.path.join(
      self.source_images_root, diagonal, '*.png'))
    return source_image_paths

  def _process_button(self, bg_image, image_info, size, scale):
    real_size = [d*scale for d in image_info['size']]
    image = Image.open(image_info['path']).resize(real_size).convert(mode='RGBA')
    actual_offset = [
      self._calculate_actual_offset(o, s, scale)
      for (o, s) in zip(image_info['offset'], size)
    ]

    image_data = list(image.getdata())
    for i, pixel in enumerate(image_data):
      image_data[i] = pixel[:3] + (image_info['opacity'], ) if pixel[3] > 0 else pixel
    image.putdata(image_data)

    bg_image.paste(image, box=actual_offset, mask=image)

  def _calculate_actual_offset(self, synth_offset, bg_image_size, scale):
     """'+' because `synth_offset` is less than zero and will decrease value on sum."""
     scaled_offset = synth_offset * scale
     return scaled_offset if synth_offset > 0 else bg_image_size + scaled_offset


class SourceImageGenerator:
  source_images_root = '../Screenshots for app store/Кандидаты на скрины из notion'
  destination_images_root = '../Screenshots for app store/sources'
  diagonals_to_ppi_scale_width_height = AppStorePreviewImageGenerator.diagonals_to_ppi_scale_width_height

  def generate_image(self, source_image_path, diagonal):
      image = Image.open(source_image_path)
      _, _, *size = self.diagonals_to_ppi_scale_width_height[diagonal]
      x_scale, y_scale = [size[i] / image.size[i] for i in range(len(size))]
      scale = x_scale if x_scale >= y_scale else y_scale
      image = image.resize([round(i*scale) for i in image.size])
      x_offset = abs(size[0]-image.size[0])
      image = image.crop((x_offset, 0, size[0]+x_offset, size[1]))

      return image

  def generate_source_image_paths(self, diagonal):
    source_image_paths = glob.glob(os.path.join(self.source_images_root, '*.png'))
    return source_image_paths

In [14]:
# ImagesGenerator(SourceImageGenerator()).generate_images()
ImagesGenerator(AppStorePreviewImageGenerator()).generate_images()

[<PIL.Image.Image image mode=RGB size=2208x1242 at 0x1048D9908>,
 <PIL.Image.Image image mode=RGB size=2208x1242 at 0x104915780>,
 <PIL.Image.Image image mode=RGB size=2208x1242 at 0x1049157B8>,
 <PIL.Image.Image image mode=RGB size=2208x1242 at 0x1048F1FD0>,
 <PIL.Image.Image image mode=RGB size=2208x1242 at 0x1048E17F0>,
 <PIL.Image.Image image mode=RGB size=2208x1242 at 0x104915E48>,
 <PIL.Image.Image image mode=RGB size=2208x1242 at 0x1048D9630>,
 <PIL.Image.Image image mode=RGB size=2208x1242 at 0x1048D9A58>,
 <PIL.Image.Image image mode=RGB size=2208x1242 at 0x104915C88>,
 <PIL.Image.Image image mode=RGB size=2208x1242 at 0x1048A0B38>,
 <PIL.Image.Image image mode=RGB size=2688x1242 at 0x1048E1240>,
 <PIL.Image.Image image mode=RGB size=2688x1242 at 0x1048E1DA0>,
 <PIL.Image.Image image mode=RGB size=2688x1242 at 0x1048E16D8>,
 <PIL.Image.Image image mode=RGB size=2688x1242 at 0x1048E1898>,
 <PIL.Image.Image image mode=RGB size=2688x1242 at 0x1048D9B00>,
 <PIL.Image.Image image m